# Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1: Marta Castillo Galán
*   Alumno 2: Unai Marín Etxebarria
*   Alumno 3: Laura Molinos Mayo
*   Alumno 4: Raúl Murillo Gallego






---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será _SpaceInvaders-v0_ y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será alcanzado cuando el agente consiga una **media de recompensa por encima de 20 puntos en modo test**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

In [79]:
#if IN_COLAB:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install keras-rl2==1.0.5
  #%pip install tensorflow==2.8
#else:
  #%pip install gym==0.17.3
  #%pip install git+https://github.com/Kojoley/atari-py.git
  #%pip install pyglet==1.5.0
  #%pip install h5py==3.1.0
  #%pip install Pillow==9.5.0
  #%pip install keras-rl2==1.0.5
  #%pip install Keras==2.2.4
  #%pip install tensorflow==2.5.3
  ##%pip install torch==2.0.1
  #%pip install agents==1.4.0

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [80]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

#### Configuración base

In [81]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [82]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

1. Implementación de la red neuronal

In [83]:

from tensorflow.keras.layers import  Dropout

def build_model(window_length, input_shape, nb_actions):
    model = Sequential()
    model.add(Permute((2, 3, 1), input_shape=(window_length,) + input_shape))  # (C, H, W) → (H, W, C)

    # Convolucional 1
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 2
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(Activation('relu'))
    
    # Convolucional 3 (ligero cambio en tamaño del kernel)
    model.add(Conv2D(512, (3, 3), strides=(1, 1), padding='same'))
    model.add(Activation('relu'))

    model.add(Flatten())

    # Capa densa + regularización
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))  # 25% dropout

    # Salida
    model.add(Dense(nb_actions))
    model.add(Activation('linear'))
    
    return model


Con el objetivo de crear una predisposición del agente para disparar, diseñamos la siguiente policy. Lo que hacemos es que durante un tiempo, o sea, la cantidad de steps nb_steps_bias, ejecutamos la acción de disparar el 30% de las veces. El resto, utilizamos un criterio voraz. Veamos.

In [84]:
from rl.policy import Policy
import numpy as np
import random

class FireBiasPolicy(Policy):
    def __init__(self, fire_action_idx, nb_steps_bias=10000):
        self.fire_action = fire_action_idx
        self.step = 0
        self.nb_steps_bias = nb_steps_bias

    def select_action(self, q_values):
        self.step += 1

        if self.step < self.nb_steps_bias:
            # Alta probabilidad de disparar (por ejemplo 30%)
            if random.random() < 0.3:
                return self.fire_action

        # Si no, acción greedy
        return np.argmax(q_values)


2. Implementación de la solución DQN

In [85]:
# Preparamos memoria y política
memory = SequentialMemory(limit=100000, window_length=WINDOW_LENGTH)
#policy = EpsGreedyQPolicy()  # Exploración simple
policy = FireBiasPolicy(fire_action_idx=1, nb_steps_bias=20000)
# Creamos el modelo
model = build_model(WINDOW_LENGTH, INPUT_SHAPE, nb_actions)
print(model.summary())

# Creamos el procesador
processor = AtariProcessor()
print(env.unwrapped.get_action_meanings())

#Creamos el agente
dqn = DQNAgent(model=model,
               nb_actions=nb_actions,
               policy=policy,
               memory=memory,
               processor=processor,
               nb_steps_warmup=10000,
               gamma=0.99,
               target_model_update=10000,
               train_interval=4,
               delta_clip=1.0)

dqn.compile(Adam(learning_rate=1e-4), metrics=['mae'])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_13 (Permute)         (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 21, 21, 32)        8224      
_________________________________________________________________
activation_35 (Activation)   (None, 21, 21, 32)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 11, 11, 64)        32832     
_________________________________________________________________
activation_36 (Activation)   (None, 11, 11, 64)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 11, 11, 512)       295424    
_________________________________________________________________
activation_37 (Activation)   (None, 11, 11, 512)      

In [86]:
# Entrenamiento inicial
# (0.0.0.2) Pasamos de 5000 a 20.000 pasos
dqn.fit(env, nb_steps=20000, visualize=False, verbose=2)

Training for 20000 steps ...


C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   496/20000: episode: 1, duration: 12.781s, episode steps: 496, steps per second:  39, episode reward:  7.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 1.629 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  1581/20000: episode: 2, duration: 21.363s, episode steps: 1085, steps per second:  51, episode reward: 12.000, mean reward:  0.011 [ 0.000,  1.000], mean action: 1.624 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2231/20000: episode: 3, duration: 12.778s, episode steps: 650, steps per second:  51, episode reward: 13.000, mean reward:  0.020 [ 0.000,  1.000], mean action: 1.680 [0.000, 5.000],  loss: --, mae: --, mean_q: --
  2874/20000: episode: 4, duration: 12.527s, episode steps: 643, steps per second:  51, episode reward: 14.000, mean reward:  0.022 [ 0.000,  1.000], mean action: 1.585 [1.000, 5.000],  loss: --, mae: --, mean_q: --
  3946/20000: episode: 5, duration: 24.730s, episode steps: 1072, steps per second:  43, episode reward: 14.000, mean reward:  0.013 [ 

C:\Users\Unai\anaconda3\envs\miar_rl\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


 10464/20000: episode: 13, duration: 85.001s, episode steps: 1476, steps per second:  17, episode reward: 27.000, mean reward:  0.018 [ 0.000,  1.000], mean action: 1.631 [0.000, 5.000],  loss: 0.009782, mae: 0.019967, mean_q: 0.044438
 11270/20000: episode: 14, duration: 95.172s, episode steps: 806, steps per second:   8, episode reward: 14.000, mean reward:  0.017 [ 0.000,  1.000], mean action: 1.700 [0.000, 5.000],  loss: 0.008654, mae: 0.018028, mean_q: 0.035967
 11959/20000: episode: 15, duration: 82.005s, episode steps: 689, steps per second:   8, episode reward: 13.000, mean reward:  0.019 [ 0.000,  1.000], mean action: 2.440 [0.000, 5.000],  loss: 0.006701, mae: 0.014688, mean_q: 0.030741
 12377/20000: episode: 16, duration: 48.964s, episode steps: 418, steps per second:   9, episode reward:  6.000, mean reward:  0.014 [ 0.000,  1.000], mean action: 2.234 [1.000, 5.000],  loss: 0.005849, mae: 0.013778, mean_q: 0.028351
 13431/20000: episode: 17, duration: 123.635s, episode step

In [87]:
#Evaluación en modo test
scores = dqn.test(env, nb_episodes=10, visualize=False)
print("Media de recompensa:", np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 16.000, steps: 1007
Episode 2: reward: 12.000, steps: 865
Episode 3: reward: 4.000, steps: 399
Episode 4: reward: 12.000, steps: 824
Episode 5: reward: 12.000, steps: 861
Episode 6: reward: 11.000, steps: 825
Episode 7: reward: 13.000, steps: 879
Episode 8: reward: 17.000, steps: 969
Episode 9: reward: 12.000, steps: 883
Episode 10: reward: 12.000, steps: 875
Media de recompensa: 12.1


In [88]:
#Guardado de pesos
dqn.save_weights('dqn_spaceinvaders_weights_firebias.h5f', overwrite=True)

In [89]:
## Testing part to calculate the mean reward
#weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
#dqn.load_weights(weights_filename)
#dqn.test(env, nb_episodes=10, visualize=False)

3. Justificación de los parámetros seleccionados y de los resultados obtenidos

---